In [1]:
import gurobipy as gp
import pandas as pd
import numpy as np
from gurobipy import GRB
import importlib
import matplotlib.pyplot as plt
import matplotlib
import sys
sys.path.append('/Users/boosman/Documents/2_Econometrie/Master/Thesis/Non_ADMM_OPF')

from Devices_NA import CHP, Generator, Renewable, TransmissionLine, ThermalLoad, FixedLoad, FixedLoadTest, Storage
from Connections_NA import EConnection, HConnection
from Networks_NA import Network
from Potential_Devices import addBudgetConstraint, PotentialStorage, PotentialTransmissionLine, PotentialRenewable
from Devices_Heuristic import HeuristicStorage, HeuristicTransmissionLine, HeuristicRenewable


In [2]:
import Devices_NA
import Connections_NA
import Networks_NA
import Potential_Devices
import importlib
import Devices_Heuristic
importlib.reload(Devices_NA)
importlib.reload(Connections_NA)
importlib.reload(Networks_NA)
importlib.reload(Potential_Devices)
importlib.reload(Devices_Heuristic)

from Connections_NA import EConnection, HConnection
from Networks_NA import Network
from Devices_NA import CHP, Generator, Renewable, TransmissionLine, ThermalLoad, FixedLoad, FixedLoadTest, Storage, PowerDissipation
from Potential_Devices import addBudgetConstraint, PotentialStorage, PotentialTransmissionLine, PotentialRenewable
from Devices_Heuristic import HeuristicStorage, HeuristicTransmissionLine, HeuristicRenewable


# Benchmark

In [3]:
#initiate the model and the time vector

m = gp.Model()
m.setParam('OutputFlag', 0)
T = list(range(8760))


##############################
# NET 1 Devices: 
LE_hh_1= FixedLoad(T, m,  'Households' , 30000)
LE_wc_1= FixedLoad(T, m, 'Weighted_Commercial' , 35000)
G_1 = Generator(T, m,  power_max=12, power_min=0, ramp_min = -0.25, ramp_max = 0.25, operating_point = 3, alpha=1, beta=40, gamma = 1)
PD_1 = PowerDissipation(T, m)

##############################
# NET 2 Devices: 
LE_hh_2= FixedLoad(T, m, 'Households' , 5000)
PD_2 = PowerDissipation(T, m)


##############################
# NET 3 Devices: 
LE_m_3= FixedLoad(T, m,  'Manufacturing' , 20000)
B_3    = Storage(T, m, discharge_max = 3, charge_max = 3, energy_max = 60)
PD_3 = PowerDissipation(T, m)


##############################
# NET 4 Devices: 
B_4    = Storage(T, m, discharge_max = 2, charge_max = 2, energy_max = 30)
LE_hh_4= FixedLoad(T, m, 'Households' , 6000)
PD_4 = PowerDissipation(T, m)


##############################
# NET 5 Devices: 
G_5 = Generator(T, m,  power_max=20, power_min=0, ramp_min = -0.1, ramp_max = 0.1, operating_point = 4, alpha=0.4, beta=35, gamma = 1)
PV_5 = Renewable(T, m, technology  = 'PV', install_cap = 11)
PD_5 = PowerDissipation(T, m)


##############################
# NET 6 Devices: 
PV_6 = Renewable(T, m, technology  = 'PV', install_cap = 6)
W_6 = Renewable(T, m, technology  = 'Wind', install_cap = 12)
PD_6 = PowerDissipation(T, m)

##############################
# NET 7 Devices: 
LE_wc_7= FixedLoad(T, m, 'Weighted_Commercial' , 4000)
LE_m_7= FixedLoad(T, m, 'Manufacturing' , 5000)
PD_7 = PowerDissipation(T, m)


##############################
# NET 8 Devices: 
LE_hh_8= FixedLoad(T, m, 'Households' , 10000)
LE_wc_8= FixedLoad(T, m, 'Weighted_Commercial' , 4000)
PV_8 = Renewable(T, m, technology  = 'PV', install_cap = 2)
PD_8 = PowerDissipation(T, m)

##############################
# NET 9 Devices: 
W_9 = Renewable(T, m, technology  = 'Wind', install_cap = 6)
LE_m_9= FixedLoad(T, m, 'Manufacturing' , 45000)
PD_9 = PowerDissipation(T, m)


##############################
# NET 10 Devices: 
G_10 = Generator(T, m,  power_max=8, power_min=0, ramp_min = -1, ramp_max = 1, operating_point = 4, alpha=1.5, beta=55, gamma = 1)
LE_hh_10= FixedLoad(T, m, 'Households' , 20000)
LE_wc_10= FixedLoad(T, m, 'Weighted_Commercial' , 25000)
PD_10 = PowerDissipation(T, m)

##############################
# NET 11 Devices: 

############### Transmission Lines
alpha = 0.25
T1 = TransmissionLine(T, m, power_max=4, alpha = alpha)
T2 = TransmissionLine(T, m, power_max=4, alpha = alpha)
T3 = TransmissionLine(T, m, power_max=4, alpha = alpha)
T4 = TransmissionLine(T, m, power_max=4, alpha = alpha)
T5 = TransmissionLine(T, m, power_max=4, alpha = alpha)
T6 = TransmissionLine(T, m, power_max=4, alpha = alpha)
T7 = TransmissionLine(T, m, power_max=4, alpha = alpha)
T8 = TransmissionLine(T, m, power_max=4, alpha = alpha)
T9 = TransmissionLine(T, m, power_max=4, alpha = alpha)
T10= TransmissionLine(T, m, power_max=4, alpha = alpha)
T11 = TransmissionLine(T, m, power_max=4, alpha = alpha)
T12 = TransmissionLine(T, m, power_max=4, alpha = alpha)
# T13 = TransmissionLine(T, m, power_max=5)

transmisions = [T1,T2,T3,T4,T5,T6,T7,T8,T9,T10,T11,T12]





N1 = Network(T,  m, [PD_1.Econnections[0], LE_hh_1.Econnections[0], LE_wc_1.Econnections[0], G_1.Econnections[0], T1.Econnections[0], T2.Econnections[0]])
N2 = Network(T,  m, [PD_2.Econnections[0], LE_hh_2.Econnections[0], T2.Econnections[1], T3.Econnections[0]])
N3 = Network(T,  m, [PD_3.Econnections[0], LE_m_3.Econnections[0], B_3.Econnections[0], T4.Econnections[0], T5.Econnections[0]])
N4 = Network(T,  m, [PD_4.Econnections[0], LE_hh_4.Econnections[0], B_4.Econnections[0], T1.Econnections[1], T6.Econnections[0]])
N5 = Network(T,  m, [PD_5.Econnections[0], G_5.Econnections[0], PV_5.Econnections[0], T3.Econnections[1], T4.Econnections[1], T7.Econnections[0]])
N6 = Network(T,  m, [PD_6.Econnections[0], PV_6.Econnections[0], W_6.Econnections[0], T8.Econnections[0], T9.Econnections[0]])
N7 = Network(T,  m, [PD_7.Econnections[0], LE_wc_7.Econnections[0], LE_m_7.Econnections[0], T6.Econnections[1], T7.Econnections[1], T9.Econnections[1], T10.Econnections[0], T11.Econnections[0]])
N8 = Network(T,  m, [PD_8.Econnections[0], LE_hh_8.Econnections[0], LE_wc_8.Econnections[0], PV_8.Econnections[0], T5.Econnections[1], T12.Econnections[0]])
N9 = Network(T,  m, [PD_9.Econnections[0], W_9.Econnections[0], LE_m_9.Econnections[0], T8.Econnections[1], T10.Econnections[1]])
N10 = Network(T, m, [PD_10.Econnections[0], G_10.Econnections[0], LE_hh_10.Econnections[0], LE_wc_10.Econnections[0], T11.Econnections[1], T12.Econnections[1]])

nets = [N1, N2, N3, N4, N5, N6, N7, N8, N9, N10]

Set parameter Username
Academic license - for non-commercial use only - expires 2025-01-23


In [ ]:
m.optimize()
print(m.ObjVal)

In [ ]:
#initiate the model and the time vector
small_wind = 3
large_wind = 6
small_pv = 2
large_pv = 6
small_storage = 30
large_storage = 60
high_transmission_cap = 4
low_transmission_cap = 2
charge_max = 2
discharge_max = 2
alpha=0.25

price_list = { 'Storage' : {small_storage : 15, large_storage : 28}, 'Wind' :{small_wind: 37 , large_wind: 70}, 
              'PV' :{small_pv : 14, large_pv: 35},
              'Transmission' : {'Short':{low_transmission_cap: 4, high_transmission_cap : 7}, 'Long':{low_transmission_cap :7, high_transmission_cap :13}
             }}

# Defining all devices
P_W_2_3 = HeuristicRenewable(T, m, 1, price_list, 'Wind', small_wind)
P_W_2_6 = HeuristicRenewable(T, m, 1, price_list, 'Wind', large_wind)
P_W_3_3 = HeuristicRenewable(T, m, 2, price_list, 'Wind', small_wind)
P_W_3_6 = HeuristicRenewable(T, m, 2, price_list, 'Wind', large_wind)
P_W_4_3 = HeuristicRenewable(T, m, 3, price_list, 'Wind', small_wind)
P_W_4_6 = HeuristicRenewable(T, m, 3, price_list, 'Wind', large_wind)
P_W_5_3 = HeuristicRenewable(T, m, 4, price_list, 'Wind', small_wind)
P_W_5_6 = HeuristicRenewable(T, m, 4, price_list, 'Wind', large_wind)
P_W_6_3 = HeuristicRenewable(T, m, 5, price_list, 'Wind', small_wind)
P_W_6_6 = HeuristicRenewable(T, m, 5, price_list, 'Wind', large_wind)
P_W_7_3 = HeuristicRenewable(T, m, 6, price_list, 'Wind', small_wind)
P_W_7_6 = HeuristicRenewable(T, m, 6, price_list, 'Wind', large_wind)
P_W_8_3 = HeuristicRenewable(T, m, 7, price_list, 'Wind', small_wind)
P_W_8_6 = HeuristicRenewable(T, m, 7, price_list, 'Wind', large_wind)
P_W_9_3 = HeuristicRenewable(T, m, 8, price_list, 'Wind', small_wind)
P_W_9_6 = HeuristicRenewable(T, m, 8, price_list, 'Wind', large_wind)
P_W_10_3 = HeuristicRenewable(T, m, 9, price_list, 'Wind', small_wind)

P_PV_1_1 = HeuristicRenewable(T, m, 0, price_list, 'PV', small_pv)
P_PV_2_1 = HeuristicRenewable(T, m, 1, price_list, 'PV', small_pv)
P_PV_2_4 = HeuristicRenewable(T, m, 1, price_list, 'PV', large_pv)
P_PV_3_1 = HeuristicRenewable(T, m, 2, price_list, 'PV', small_pv)
P_PV_3_4 = HeuristicRenewable(T, m, 2, price_list, 'PV', large_pv)
P_PV_4_1 = HeuristicRenewable(T, m, 3, price_list, 'PV', small_pv)
P_PV_4_4 = HeuristicRenewable(T, m, 3, price_list, 'PV', large_pv)
P_PV_5_1 = HeuristicRenewable(T, m, 4, price_list, 'PV', small_pv)
P_PV_5_4 = HeuristicRenewable(T, m, 4, price_list, 'PV', large_pv)
P_PV_6_1 = HeuristicRenewable(T, m, 5, price_list, 'PV', small_pv)
P_PV_6_4 = HeuristicRenewable(T, m, 5, price_list, 'PV', large_pv)
P_PV_7_1 = HeuristicRenewable(T, m, 6, price_list, 'PV', small_pv)
P_PV_7_4 = HeuristicRenewable(T, m, 6, price_list, 'PV', large_pv)
P_PV_8_1 = HeuristicRenewable(T, m, 7, price_list, 'PV', small_pv)
P_PV_8_4 = HeuristicRenewable(T, m, 7, price_list, 'PV', large_pv)
P_PV_9_1 = HeuristicRenewable(T, m, 8, price_list, 'PV', small_pv)
P_PV_9_4 = HeuristicRenewable(T, m, 8, price_list, 'PV', large_pv)
P_PV_10_1 = HeuristicRenewable(T, m, 9, price_list, 'PV', small_pv)

P_B_1_60 = HeuristicStorage(T, m, 0, price_list, discharge_max=discharge_max, charge_max=charge_max, energy_max=small_storage)
P_B_1_120 = HeuristicStorage(T, m, 0, price_list, discharge_max=discharge_max, charge_max=charge_max, energy_max=large_storage)
P_B_2_60 = HeuristicStorage(T, m, 1, price_list, discharge_max=discharge_max, charge_max=charge_max, energy_max=small_storage)
P_B_2_120 = HeuristicStorage(T, m, 1, price_list, discharge_max=discharge_max, charge_max=charge_max, energy_max=large_storage)
P_B_3_60 = HeuristicStorage(T, m, 2, price_list, discharge_max=discharge_max, charge_max=charge_max, energy_max=small_storage)
P_B_3_120 = HeuristicStorage(T, m, 2, price_list, discharge_max=discharge_max, charge_max=charge_max, energy_max=large_storage)
P_B_4_60 = HeuristicStorage(T, m, 3, price_list, discharge_max=discharge_max, charge_max=charge_max, energy_max=small_storage)
P_B_4_120 = HeuristicStorage(T, m, 3, price_list, discharge_max=discharge_max, charge_max=charge_max, energy_max=large_storage)
P_B_5_60 = HeuristicStorage(T, m, 4, price_list, discharge_max=discharge_max, charge_max=charge_max, energy_max=small_storage)
P_B_5_120 = HeuristicStorage(T, m, 4, price_list, discharge_max=discharge_max, charge_max=charge_max, energy_max=large_storage)
P_B_6_60 = HeuristicStorage(T, m, 5, price_list, discharge_max=discharge_max, charge_max=charge_max, energy_max=small_storage)
P_B_6_120 = HeuristicStorage(T, m, 5, price_list, discharge_max=discharge_max, charge_max=charge_max, energy_max=large_storage)
P_B_7_60 = HeuristicStorage(T, m, 6, price_list, discharge_max=discharge_max, charge_max=charge_max, energy_max=small_storage)
P_B_7_120 = HeuristicStorage(T, m, 6, price_list, discharge_max=discharge_max, charge_max=charge_max, energy_max=large_storage)
P_B_8_60 = HeuristicStorage(T, m, 7, price_list, discharge_max=discharge_max, charge_max=charge_max, energy_max=small_storage)
P_B_8_120 = HeuristicStorage(T, m, 7, price_list, discharge_max=discharge_max, charge_max=charge_max, energy_max=large_storage)
P_B_9_60 = HeuristicStorage(T, m, 8, price_list, discharge_max=discharge_max, charge_max=charge_max, energy_max=small_storage)
P_B_9_120 = HeuristicStorage(T, m, 8, price_list, discharge_max=discharge_max, charge_max=charge_max, energy_max=large_storage)
P_B_10_60 = HeuristicStorage(T, m, 9, price_list, discharge_max=discharge_max, charge_max=charge_max, energy_max=small_storage)
P_B_10_120 = HeuristicStorage(T, m, 9, price_list, discharge_max=discharge_max, charge_max=charge_max, energy_max=large_storage)

P_T_1_high = HeuristicTransmissionLine(T, m, 0, 5, price_list, length='Long', power_max=high_transmission_cap, alpha=alpha)
P_T_1_low = HeuristicTransmissionLine(T, m, 0, 5, price_list, length='Long', power_max=low_transmission_cap)
P_T_2_high = HeuristicTransmissionLine(T, m, 1, 3, price_list, length='Long', power_max=high_transmission_cap, alpha=alpha)
P_T_2_low = HeuristicTransmissionLine(T, m, 1, 3, price_list, length='Long', power_max=low_transmission_cap)
P_T_3_high = HeuristicTransmissionLine(T, m, 1, 2, price_list, length='Short', power_max=high_transmission_cap, alpha=alpha)
P_T_3_low = HeuristicTransmissionLine(T, m, 1, 2, price_list, length='Short', power_max=low_transmission_cap)
P_T_4_high = HeuristicTransmissionLine(T, m, 3, 5, price_list, length='Long', power_max=high_transmission_cap, alpha=alpha)
P_T_4_low = HeuristicTransmissionLine(T, m, 3, 5, price_list, length='Long', power_max=low_transmission_cap)
P_T_5_high = HeuristicTransmissionLine(T, m, 3, 4, price_list, length='Short', power_max=high_transmission_cap, alpha=alpha)
P_T_5_low = HeuristicTransmissionLine(T, m, 3, 4, price_list, length='Short', power_max=low_transmission_cap)
P_T_6_high = HeuristicTransmissionLine(T, m, 4, 9, price_list, length='Long', power_max=high_transmission_cap, alpha=alpha)
P_T_6_low = HeuristicTransmissionLine(T, m, 4, 9, price_list, length='Long', power_max=low_transmission_cap)
P_T_7_high = HeuristicTransmissionLine(T, m, 4, 7, price_list, length='Long', power_max=high_transmission_cap, alpha=alpha)
P_T_7_low = HeuristicTransmissionLine(T, m, 4, 7, price_list, length='Long', power_max=low_transmission_cap)
P_T_8_high = HeuristicTransmissionLine(T, m, 5, 9, price_list, length='Long', power_max=high_transmission_cap, alpha=alpha)
P_T_8_low = HeuristicTransmissionLine(T, m, 5, 9, price_list, length='Long', power_max=low_transmission_cap)
P_T_9_high = HeuristicTransmissionLine(T, m, 8, 9, price_list, length='Short', power_max=high_transmission_cap, alpha=alpha)
P_T_9_low = HeuristicTransmissionLine(T, m, 8, 9, price_list, length='Short', power_max=low_transmission_cap)

# Adjusting potential items list
potential_items = [
    P_W_2_3, P_W_2_6, P_W_3_3, P_W_3_6, P_W_4_3, P_W_4_6, P_W_5_3, P_W_5_6, P_W_6_3, P_W_6_6, P_W_7_3, P_W_7_6, P_W_8_3, P_W_8_6, P_W_9_3, P_W_9_6, P_W_10_3,
    P_PV_1_1, P_PV_2_1, P_PV_2_4, P_PV_3_1, P_PV_3_4, P_PV_4_1, P_PV_4_4, P_PV_5_1, P_PV_5_4, P_PV_6_1, P_PV_6_4, P_PV_7_1, P_PV_7_4, P_PV_8_1, P_PV_8_4, P_PV_9_1, P_PV_9_4, P_PV_10_1,
    P_B_1_60, P_B_1_120, P_B_2_60, P_B_2_120, P_B_3_60, P_B_3_120, P_B_4_60, P_B_4_120, P_B_5_60, P_B_5_120, P_B_6_60, P_B_6_120, P_B_7_60, P_B_7_120, P_B_8_60, P_B_8_120, P_B_9_60, P_B_9_120, P_B_10_60, P_B_10_120,
    P_T_1_high, P_T_1_low, P_T_2_high, P_T_2_low, P_T_3_high, P_T_3_low, P_T_4_high, P_T_4_low, P_T_5_high, P_T_5_low, P_T_6_high, P_T_6_low, P_T_7_high, P_T_7_low, P_T_8_high, P_T_8_low, P_T_9_high, P_T_9_low
]



In [ ]:
lower_bound = 4829083

benchmark_cost = 7357028
# object_1 = 7345764
# objec_2 = 7542120
# object_3 = 7805264
# object_4 = 7690623

def createDeviceValue(potential_items, benchmark_cost, nets):
    results = []
    net_from_previous = 100
    net_to_previous = 100
    group_prev  = 100 
    for p in potential_items: 
        print(p.name)
        if p.name in ["Renewable PV", "Renewable Wind"]:
            name = p.name
            net_number = p.net_from
            group = p.net_from
            price = p.investment_cost
            capacity =  p.install_cap
            
            effected_nets = nets[net_number]
            effected_nets.addEconnection(p.Econnections[0])

        #     # optimize
            m.optimize()
            total_cost = m.ObjVal
            
            total_saving  = benchmark_cost - total_cost
            effected_nets.removeEconnection(p.Econnections[0])
            results.append((p, name, group, capacity, price, total_saving))
    
        elif p.name == "Storage":
            name = p.name
            net_number = p.net_from
            group = p.net_from + 10
            price = p.investment_cost
            capacity =  p.energy_max
            
            effected_nets = nets[net_number]
            effected_nets.addEconnection(p.Econnections[0])
            
            # optimize
            m.optimize()
            total_cost = m.ObjVal
            
            total_saving  = benchmark_cost - total_cost
            effected_nets.removeEconnection(p.Econnections[0])
            results.append((p, name, group, capacity, price, total_saving))
            
        elif p.name == "Transmission Line":
            name = p.name
            net_number_from = p.net_from
            net_number_to = p.net_to
            net_from = nets[net_number_from]
            net_to = nets[net_number_to]
            price = p.investment_cost
            capacity =  p.power_max
            net_from.addEconnection(p.Econnections[0])
            net_to.addEconnection(p.Econnections[1])
            
            # optimize
            m.optimize()
            total_cost = m.ObjVal
            total_saving  = benchmark_cost - total_cost
            
            if net_to == net_to_previous and net_from == net_from_previous:
                group = group_prev
            else:
                max_group = max(results, key=lambda x: x[2])  
                max_group_value = max_group[2]
                group = max(21, max_group_value + 1)
            
            results.append((p, name, group, capacity, price, total_saving))
            net_from.removeEconnection(p.Econnections[0])
            net_to.removeEconnection(p.Econnections[1])
            net_from_previous = net_from
            net_to_previous = net_to
            group_prev = group
        else: 
            print("Something went Wrong")

    return results
    


In [ ]:
#groups is list of integers in ascending order without gaps

def getRow(lists, row):
    res = []
    for l in lists:
        for i in range(len(l)):
            if i==row:
                res.append(l[i])
    return res

def multipleChoiceKnapsack(W, weights, values, groups):
    n = len(values)
    K = [[0 for x in range(W + 1)] for x in range(n + 1)]

    for w in range(W + 1):
        for i in range(n + 1):
            if i == 0 or w == 0:
                K[i][w] = 0
            elif weights[i - 1] <= w:
                sub_max = 0
                prev_group = groups[i - 1] - 1
                sub_K = getRow(K, w - weights[i - 1])
                for j in range(n + 1):
                    if groups[j - 1] == prev_group and sub_K[j] > sub_max:
                        sub_max = sub_K[j]
                K[i][w] = max(sub_max + values[i - 1], K[i - 1][w])
            else:
                K[i][w] = K[i - 1][w]
        if w % 10 == 0:
            print(str(w/W*100) + " %")

    # stores the result of Knapsack
    res = K[n][W]
    sol = []

    w = W
    for i in range(n, 0, -1):
        if res <= 0:
            break
        # either the result comes from the
        # top (K[i-1][w]) or from (val[i-1]
        # + K[i-1] [w-wt[i-1]]) as in Knapsack
        # table. If it comes from the latter
        # one/ it means the item is included.
        if res == K[i - 1][w]:
            continue
        else:

            # This item is included.
            sol.append(i - 1)
            print(i - 1)

            #sub_max = 0
            #prev_group = groups[i - 1] - 1
            #sub_K = getRow(K, w - weights[i - 1])
            #for j in range(n + 1):
            #    if groups[j - 1] == prev_group and sub_K[j] > sub_max:
            #        sub_max = sub_K[j]

            # Since this weight is included
            # its value is deducted
            res = res - values[i - 1]
            w = w - weights[i - 1]

    return K[n][W], sol


#Example
budget = 300
# print(multipleChoiceKnapsack(W, weights, values, groups)) #220

In [ ]:
def solutionRetriever(sorted_list, index_list, nets):
    filtered_tuples = [sorted_list[idx] for idx in index_list]

    for i in filtered_tuples: 
        device = i[0]
        if device.name in ["Renewable PV", "Renewable Wind"]:
            net_number = device.net_from
            effected_nets = nets[net_number]
            effected_nets.addEconnection(device.Econnections[0])
            
        elif device.name == "Storage":
            net_number = device.net_from
            effected_nets = nets[net_number]
            effected_nets.addEconnection(device.Econnections[0])
        
        elif device.name == "Transmission Line":
            net_number_from = device.net_from
            net_number_to = device.net_to
            net_from = nets[net_number_from]
            net_to = nets[net_number_to]
            net_from.addEconnection(device.Econnections[0])
            net_to.addEconnection(device.Econnections[1])

    m.optimize()
    return m.ObjVal

In [ ]:
### Dynamic Programming

result = createDeviceValue(potential_items, benchmark_cost, nets)
sorted_by_group = sorted(result, key=lambda x: x[2])
group = [i[2] for i in sorted_list]
value = [i[5] for i in sorted_list]
price = [i[4] for i in sorted_list] 
budget = 300
meaningless, items_selected = multipleChoiceKnapsack(budget, price, value, group)
solution_value = solutionRetriever(sorted_list, items_selected, nets)

In [ ]:
def greedyHeuristic(potential_items, benchmark_cost, nets, budget):
    termination = False
    potential_items = potential_items
    residual_old = budget
    selected_items = [] 
    while termination == False: 
        results =  createDeviceValue(potential_items, benchmark_cost, nets)
        sortedROI = sorted(results, key=lambda x: x[5] / x[4])
        for item in sortedROI:
            if item[4] <= residual:
                selected_items.append(item[0])
                potential_items.remove(item[0])
                residual = residual_old - item[4]
                print(residual)
                break

        if residual_old  == residual:
            termination == True
        else:
            residual_old = residual
            
    return selected_items
            
                
            


In [ ]:
#GReedy Approach
budget = 300

selected_items = greedyHeuristic(potential_items, benchmark_cost, nets, budget)

In [ ]:
selected_items


In [ ]:
group = [res[2] for res in sorted_list]
value = [-res[5] for res in sorted_list]
price = [res[4] for res in sorted_list] 

In [ ]:
lower_bound = 4829083

benchmark_cost = 7834980
object_1 = 7345764
objec_2 = 7542120
object_3 = 7805264
object_4 = 7629665


In [ ]:
value

In [ ]:
len(sorted_list)

In [ ]:
a, b = multipleChoiceKnapsack(budget, price, value, group)

In [ ]:
b

In [ ]:
devices_selected = [sorted_list[idx] for idx in b]

In [ ]:
for i in devices_selected:
    print(f"{i[0].name} at net {i[0].net_from}")

In [ ]:
sol = solutionRetriever(sorted_list, b, nets)

In [ ]:
m.optimize()
if m.status == GRB.OPTIMAL:
    print("Optimal solution found.")
elif m.status == GRB.INFEASIBLE:
    print("Model is infeasible.")
elif m.status == GRB.UNBOUNDED:
    print("Model is unbounded.")
elif m.status == GRB.INF_OR_UNBD:
    print("Model is either infeasible or unbounded.")
else:
    print("Optimization was stopped with status", m.status)
N1.updateDual()
N2.updateDual()
N3.updateDual()
N4.updateDual()
N5.updateDual()
N6.updateDual()
N7.updateDual()
N8.updateDual()
N9.updateDual()
N10.updateDual()

In [ ]:
def average(lst):
    return sum(lst) / len(lst)

def printDuals(nets):
    i=1
    for n in nets:
        print(f"Net {i} - avg Dual: {average(n.dual)}")
        i +=1
        
def printMaxCapacity(transmissions, epsilon):
    i =1 
    for t in transmissions:
        count = sum(1 for value in t.Econnections[0].powerValues if abs(value - t.power_max) <= epsilon)
        print(f"Transmission line {i} is {count/87.60:.2f}% of the time opertaing at its maximum capacity")
        i+=1

In [ ]:
printDuals(nets)

In [ ]:
printMaxCapacity(transmisions, 0.1)

In [ ]:
def renewablePowerShed(power_dissipation_devices):
    return sum([sum(sublist.Econnection[0].powerValues) for sublist in power_dissipation_devices])        

def unusedHeat(heat_dissipation_devices):
    return sum([sum(sublist.Hconnection[0].powerValues) for sublist in heat_dissipation_devices])

def emissions(generators, boilers, alpha):
    #alpha is the emission per mwh output electricity
    total = 0        
    total += sum([sum(sublist.Econnection[0].powerValues) for sublist in generators]) * alpha
    total += sum([sum(sublist.Hconnection[0].powerValues) for sublist in boilers])    * alpha * 0.444 
    return total
    


In [ ]:
#initiate the model and the time vector

m = gp.Model()
log_file = "gurobi_log.log"
m.setParam(GRB.Param.LogFile, log_file)
# m.setParam('TimeLimit', 60)          # Terminate after 60 seconds
# m.setParam('MIPGap', 0.01)           # Terminate when within 1% of optimal solution
# m.setParam('Presolve', 2)            # Aggressive presolve
# m.setParam('Cuts', 2)                # Aggressive cuts
# m.setParam('Heuristics', 0.5)        # Increase heuristics

T = list(range(8760))


wind_capacity = 6
pv_capacity = 4
storage_capacity = 60
high_transmission_cap = 3
transmission_cap = 5

price_list = { 'Storage' : {storage_capacity : 24}, 
              'Wind' :{wind_capacity: 6}, 
              'PV' :{pv_capacity: 3},
              'Transmission' : {'Short':{transmission_cap: 15}, 'Long':{transmission_cap :24}}
             }

# P_W_1 = HeuristicRenewable(T, m, price_list, 'Wind', wind_capacity)
# P_W_2 = PotentialRenewable(T, m, price_list, 'Wind', wind_capacity)
# P_W_3 = PotentialRenewable(T, m, price_list, 'Wind', wind_capacity)
# P_W_4 = PotentialRenewable(T, m, price_list, 'Wind', wind_capacity)
# P_W_5 = PotentialRenewable(T, m, price_list, 'Wind', wind_capacity)
# P_W_6 = PotentialRenewable(T, m, price_list, 'Wind', wind_capacity)
P_W_7 = PotentialRenewable(T, m, price_list, 'Wind', wind_capacity)
# P_W_8 = PotentialRenewable(T, m, price_list, 'Wind', wind_capacity)
# P_W_9 = PotentialRenewable(T, m, price_list, 'Wind', wind_capacity)
# P_W_10 = PotentialRenewable(T, m, price_list, 'Wind', wind_capacity)

# P_PV_1 = PotentialRenewable(T, m, price_list, 'PV', pv_capacity)
# P_PV_2 = PotentialRenewable(T, m, price_list, 'PV', pv_capacity)
# P_PV_3 = PotentialRenewable(T, m, price_list, 'PV', pv_capacity)
# P_PV_4 = PotentialRenewable(T, m, price_list, 'PV', pv_capacity)
# P_PV_5 = PotentialRenewable(T, m, price_list, 'PV', pv_capacity)
# P_PV_6 = PotentialRenewable(T, m, price_list, 'PV', pv_capacity)
# P_PV_7 = PotentialRenewable(T, m, price_list, 'PV', pv_capacity)
# P_PV_8 = PotentialRenewable(T, m, price_list, 'PV', pv_capacity)
P_PV_9 = PotentialRenewable(T, m, price_list, 'PV', pv_capacity)
# P_PV_10 = PotentialRenewable(T, m, price_list, 'PV', pv_capacity)

# P_B_1 = PotentialStorage(T, m, price_list, discharge_max = 2, charge_max = 2, energy_max =storage_capacity)
# P_B_2 = PotentialStorage(T, m, price_list, discharge_max = 2, charge_max = 2, energy_max =storage_capacity)
# # P_B_3 = PotentialStorage(T, m, price_list, discharge_max = 2, charge_max = 2, energy_max =storage_capacity)
# P_B_4 = PotentialStorage(T, m, price_list, discharge_max = 2, charge_max = 2, energy_max =storage_capacity)
# P_B_5 = PotentialStorage(T, m, price_list, discharge_max = 2, charge_max = 2, energy_max =storage_capacity)
# P_B_6 = PotentialStorage(T, m, price_list, discharge_max = 2, charge_max = 2, energy_max =storage_capacity)
# P_B_7 = PotentialStorage(T, m, price_list, discharge_max = 2, charge_max = 2, energy_max =storage_capacity)
# P_B_8 = PotentialStorage(T, m, price_list, discharge_max = 2, charge_max = 2, energy_max =storage_capacity)
# P_B_9 = PotentialStorage(T, m, price_list, discharge_max = 2, charge_max = 2, energy_max =storage_capacity)
# P_B_10 = PotentialStorage(T, m, price_list,discharge_max = 2, charge_max = 2, energy_max = storage_capacity)


# P_T_1 = PotentialTransmissionLine(T, m, price_list, length = 'Long', power_max =transmission_cap)
# P_T_2 = PotentialTransmissionLine(T, m, price_list, length = 'Long', power_max =transmission_cap)
# P_T_3 = PotentialTransmissionLine(T, m, price_list, length = 'Short',power_max = mission_cap)
# P_T_4 = PotentialTransmissionLine(T, m, price_list, length = 'Long', power_max =transmission_cap)
# P_T_5 = PotentialTransmissionLine(T, m, price_list, length = 'Short',power_max = transmission_cap)
# # P_T_6 = PotentialTransmissionLine(T, m, price_list, length = 'Long', power_max =transmission_cap)
# P_T_7 = PotentialTransmissionLine(T, m, price_list, length = 'Long', power_max =transmission_cap)
# P_T_8 = PotentialTransmissionLine(T, m, price_list, length = 'Long', power_max =transmission_cap)
# P_T_9 = PotentialTransmissionLine(T, m, price_list, length = 'Short',power_max = transmission_cap)

# potential_devices = [P_W_1, P_W_2, P_W_3, P_W_4, P_W_5, P_W_6, P_W_7, P_W_8, P_W_9, P_W_10,
#                      P_PV_1, P_PV_2, P_PV_3, P_PV_4, P_PV_5, P_PV_6, P_PV_7, P_PV_8, P_PV_9, P_PV_10,
#                      P_B_1,  P_B_2, P_B_3, P_B_4, P_B_5, P_B_6, P_B_7, P_B_8, P_B_9, P_B_10, 
#                      P_T_1, P_T_5, P_T_7, P_T_8]

# potential_devices = [
#     P_W_2, P_W_3, P_W_5, P_W_7, P_W_9,
#     P_PV_1, P_PV_2, P_PV_3, P_PV_4, P_PV_9, P_PV_10,
#     P_B_2, P_B_6, P_B_8,
#     P_T_1, P_T_5, P_T_7, P_T_8
# ]

potential_devices = [
    P_W_7, P_PV_9, 
    # P_B_6, 
    # P_T_1
]

In [ ]:
P_W_7.investment_cost

In [ ]:

##############################
# NET 1 Devices: 
LE_hh_1= FixedLoad(T, m,  'Households' , 30000)
LE_wc_1= FixedLoad(T, m, 'Weighted_Commercial' , 35000)
G_1 = Generator(T, m,  power_max=12, power_min=0, ramp_min = -1, ramp_max = 1, operating_point = 3, alpha=1, beta=40, gamma = 1)
PD_1 = PowerDissipation(T, m)

##############################
# NET 2 Devices: 
LE_hh_2= FixedLoad(T, m, 'Households' , 5000)
PD_2 = PowerDissipation(T, m)


##############################
# NET 3 Devices: 
LE_m_3= FixedLoad(T, m,  'Manufacturing' , 20000)
B_3    = Storage(T, m, discharge_max = 3, charge_max = 3, energy_max = 60)
PD_3 = PowerDissipation(T, m)


##############################
# NET 4 Devices: 
B_4    = Storage(T, m, discharge_max = 2, charge_max = 2, energy_max = 30)
LE_hh_4= FixedLoad(T, m, 'Households' , 6000)
PD_4 = PowerDissipation(T, m)


##############################
# NET 5 Devices: 
G_5 = Generator(T, m,  power_max=20, power_min=0, ramp_min = -1, ramp_max = 1, operating_point = 4, alpha=0.4, beta=35, gamma = 1)
PV_5 = Renewable(T, m, technology  = 'PV', install_cap = 7)
PD_5 = PowerDissipation(T, m)


##############################
# NET 6 Devices: 
PV_6 = Renewable(T, m, technology  = 'PV', install_cap = 6)
W_6 = Renewable(T, m, technology  = 'Wind', install_cap = 9)
PD_6 = PowerDissipation(T, m)

##############################
# NET 7 Devices: 
LE_wc_7= FixedLoad(T, m, 'Weighted_Commercial' , 4000)
LE_m_7= FixedLoad(T, m, 'Manufacturing' , 5000)
PD_7 = PowerDissipation(T, m)


##############################
# NET 8 Devices: 
LE_hh_8= FixedLoad(T, m, 'Households' , 10000)
LE_wc_8= FixedLoad(T, m, 'Weighted_Commercial' , 4000)
PV_8 = Renewable(T, m, technology  = 'PV', install_cap = 2)
PD_8 = PowerDissipation(T, m)

##############################
# NET 9 Devices: 
W_9 = Renewable(T, m, technology  = 'Wind', install_cap = 3)
LE_m_9= FixedLoad(T, m, 'Manufacturing' , 45000)
PD_9 = PowerDissipation(T, m)


##############################
# NET 10 Devices: 
G_10 = Generator(T, m,  power_max=8, power_min=0, ramp_min = -0.1, ramp_max = 0.1, operating_point = 4, alpha=1.5, beta=55, gamma = 1)
LE_hh_10= FixedLoad(T, m, 'Households' , 20000)
LE_wc_10= FixedLoad(T, m, 'Weighted_Commercial' , 25000)
PD_10 = PowerDissipation(T, m)

##############################
# NET 11 Devices: 

############### Transmission Lines
T1 = TransmissionLine(T, m,alpha = 0.01 ,power_max=5)
T2 = TransmissionLine(T, m,alpha = 0.02, power_max=2)
T3 = TransmissionLine(T, m,alpha = 0.03 ,power_max=2)
T4 = TransmissionLine(T, m,alpha = 0.04, power_max=3)
T5 = TransmissionLine(T, m,alpha = 0.05, power_max=3)
T6 = TransmissionLine(T, m,alpha = 0.06 ,power_max=5)
T7 = TransmissionLine(T, m,alpha = 0.07 ,power_max=5)
T8 = TransmissionLine(T, m,alpha = 0.08, power_max=5)
T9 = TransmissionLine(T, m,alpha = 0.09 ,power_max=5)
T10= TransmissionLine(T, m,alpha = 0.10, power_max=5)
T11 = TransmissionLine(T, m,alpha = 0.11, power_max=5)
T12 = TransmissionLine(T, m,alpha = 0.12 ,power_max=5)
# T13 = TransmissionLine(T, m, power_max=5)

transmisions = [T1,T2,T3,T4,T5,T6,T7,T8,T9,T10,T11,T12]


In [ ]:
pot_1 = [
        # P_PV_1.Econnections[0], 
        # P_T_1.Econnections[0], 
        ]

pot_2 = [
        # P_W_2.Econnections[0],
        # P_PV_2.Econnections[0],
        # P_B_2.Econnections[0],
        ]

pot_3 = [
        # P_W_3.Econnections[0],
        # P_PV_3.Econnections[0],
        ]

pot_4 = [
        # P_PV_4.Econnections[0],
        # P_T_5.Econnections[0]
        ]

pot_5 = [
        # P_W_5.Econnections[0],
        # P_T_5.Econnections[1],
        # P_T_7.Econnections[0],
        ]

pot_6 = [
        # P_B_6.Econnections[0],
        # P_T_1.Econnections[1],
        # P_T_8.Econnections[0]
        ]

pot_7 = [
        P_W_7.Econnections[0],
        ]

pot_8 = [
        # P_B_8.Econnections[0],
        # P_T_7.Econnections[1],
        ]

pot_9 = [
        # P_W_9.Econnections[0],
        P_PV_9.Econnections[0],
        ]

pot_10 = [
        # P_PV_10.Econnections[0],
        # P_T_8.Econnections[1]
         ]

In [ ]:
pot_9[0].device

In [ ]:

N1 = Network(T,  m, [PD_1.Econnections[0], LE_hh_1.Econnections[0], LE_wc_1.Econnections[0], G_1.Econnections[0], T1.Econnections[0], T2.Econnections[0]] + pot_1)
N2 = Network(T,  m, [PD_2.Econnections[0], LE_hh_2.Econnections[0], T2.Econnections[1], T3.Econnections[0]] + pot_2)
N3 = Network(T,  m, [PD_3.Econnections[0], LE_m_3.Econnections[0], B_3.Econnections[0], T4.Econnections[0], T5.Econnections[0]] + pot_3)
N4 = Network(T,  m, [PD_4.Econnections[0], LE_hh_4.Econnections[0], B_4.Econnections[0], T1.Econnections[1], T6.Econnections[0]] + pot_4)
N5 = Network(T,  m, [PD_5.Econnections[0], G_5.Econnections[0], PV_5.Econnections[0], T3.Econnections[1], T4.Econnections[1], T7.Econnections[0]] + pot_5)
N6 = Network(T,  m, [PD_6.Econnections[0], PV_6.Econnections[0], W_6.Econnections[0], T8.Econnections[0], T9.Econnections[0]] + pot_6)
N7 = Network(T,  m, [PD_7.Econnections[0], LE_wc_7.Econnections[0], LE_m_7.Econnections[0], T6.Econnections[1], T7.Econnections[1], T9.Econnections[1], T10.Econnections[0], T11.Econnections[0]] + pot_7)
N8 = Network(T,  m, [PD_8.Econnections[0], LE_hh_8.Econnections[0], LE_wc_8.Econnections[0], PV_8.Econnections[0], T5.Econnections[1], T12.Econnections[0]] + pot_8)
N9 = Network(T,  m, [PD_9.Econnections[0], W_9.Econnections[0], LE_m_9.Econnections[0], T8.Econnections[1], T10.Econnections[1]] + pot_9)
N10 = Network(T, m, [PD_10.Econnections[0], G_10.Econnections[0], LE_hh_10.Econnections[0], LE_wc_10.Econnections[0], T11.Econnections[1], T12.Econnections[1]] + pot_10)

nets = [N1, N2, N3, N4, N5, N6, N7, N8, N9, N10]

In [ ]:
addBudgetConstraint(potential_devices, m, 6)

In [ ]:
m.optimize()
if m.status == GRB.OPTIMAL:
    print("Optimal solution found.")
elif m.status == GRB.INFEASIBLE:
    print("Model is infeasible.")
elif m.status == GRB.UNBOUNDED:
    print("Model is unbounded.")
elif m.status == GRB.INF_OR_UNBD:
    print("Model is either infeasible or unbounded.")
else:
    print("Optimization was stopped with status", m.status)
# N1.updateDual()

In [ ]:
for p in potential_devices:
    print(p.z.x)

In [ ]:
def average(lst):
    return sum(lst) / len(lst)

In [ ]:
[1,2,3] +[]

In [ ]:
old_powervalues = {'T1': T1.Econnections[0].powerValues ,
                   'T2': T2.Econnections[0].powerValues,
                  'T3': T3.Econnections[0].powerValues,
                  'T4': T4.Econnections[0].powerValues,
                  'T5': T5.Econnections[0].powerValues,
                  'T6': T6.Econnections[0].powerValues,
                  'T7': T7.Econnections[0].powerValues,
                  'T8': T8.Econnections[0].powerValues,
                  'T9': T9.Econnections[0].powerValues,
                  'T10': T10.Econnections[0].powerValues,
                  'T11': T11.Econnections[0].powerValues,
                  'T12': T12.Econnections[0].powerValues,}

In [ ]:
min(PD_7.Econnections[0].powerValues)

In [ ]:
T10.Econnections[0].powerValues.index(min(T10.Econnections[0].powerValues))


In [ ]:
T10.Econnections[0].powerValues

In [ ]:
T10.Econnections[0].powerValues

In [ ]:
T8.Econnections[0].powerValues

In [ ]:
T8.Econnections[0].powerValues

In [ ]:
T9.Econnections[0].powerValues

In [ ]:
T9.Econnections[0].powerValues